In [1]:
!pip install -q mediapipe

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<

In [2]:
import mediapipe as mp
import numpy as np
import cv2
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm

mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh
mp_hands = mp.solutions.hands

COORDINATES = 3  # (x, y, z) for each landmark

filtered_hand = list(range(21))

filtered_pose = [0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]

filtered_face = [10, 152, 14, 7, 8, 144, 145, 146, 153, 154, 155, 158, 159, 160, 161, 163, 362, 385, 387, 388, 390, 398, 400, 402, 417, 419, 420, 421, 423, 424, 425, 426, 427, 428, 429, 430, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449]

POSE_LANDMARKS = len(filtered_pose)
FACE_LANDMARKS = len(filtered_face)
HAND_LANDMARKS = len(filtered_hand)
TOTAL_LANDMARKS = POSE_LANDMARKS + FACE_LANDMARKS + (2 * HAND_LANDMARKS)

def pad_landmarks(landmarks, max_landmarks):
    """Pad landmarks with zeros to ensure consistent shapes."""
    if landmarks.shape[0] < max_landmarks:
        padding = np.zeros((max_landmarks - landmarks.shape[0], COORDINATES))
        landmarks = np.vstack((landmarks, padding))
    return landmarks

def extract_pose_landmarks(frame, pose):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    if results.pose_landmarks:
        landmarks = np.array([[lm.x, lm.y, lm.z] for i, lm in enumerate(results.pose_landmarks.landmark) if i in filtered_pose])
        return pad_landmarks(landmarks, POSE_LANDMARKS)
    return np.zeros((POSE_LANDMARKS, COORDINATES))

def extract_face_landmarks(frame, face_mesh):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    if results.multi_face_landmarks:
        landmarks = np.array([[lm.x, lm.y, lm.z] for face_landmarks in results.multi_face_landmarks for i, lm in enumerate(face_landmarks.landmark) if i in filtered_face])
        return pad_landmarks(landmarks, FACE_LANDMARKS)
    return np.zeros((FACE_LANDMARKS, COORDINATES))

def extract_hand_landmarks(frame, hands):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks.extend([[lm.x, lm.y, lm.z] for i, lm in enumerate(hand_landmarks.landmark) if i in filtered_hand])
        return pad_landmarks(np.array(landmarks), 2 * HAND_LANDMARKS)
    return np.zeros((2 * HAND_LANDMARKS, COORDINATES))

def process_videos(root_folder):
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, 
                                  max_num_faces=1, 
                                  min_detection_confidence=0.5, 
                                  min_tracking_confidence=0.5)
    hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    all_datasets = []  
    all_labels = [] 
    folder_count = 0
    for folder in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder)
        if os.path.isdir(folder_path):
            folder_count += 1
            for subfolder in os.listdir(folder_path):
                subfolder_path = os.path.join(folder_path, subfolder)
                if os.path.isdir(subfolder_path):
                    for sign_folder in os.listdir(subfolder_path):
                        sign_path = os.path.join(subfolder_path, sign_folder)
                        if os.path.isdir(sign_path):
                            video_files = [f for f in os.listdir(sign_path) if f.endswith('.MOV')]

                            for filename in tqdm(video_files, desc=f"Processing videos in {sign_folder}"):
                                filepath = os.path.join(sign_path, filename)
                                cap = cv2.VideoCapture(filepath)

                                while cap.isOpened():
                                    ret, frame = cap.read()
                                    if not ret:
                                        break

                                    pose_landmarks = extract_pose_landmarks(frame, pose)
                                    face_landmarks = extract_face_landmarks(frame, face_mesh)
                                    hand_landmarks = extract_hand_landmarks(frame, hands)
                                

                                    # Concatenate landmarks
                                    landmarks = np.concatenate((pose_landmarks, face_landmarks, hand_landmarks), axis=0)
                                    if landmarks.shape != (TOTAL_LANDMARKS, COORDINATES):
                                        print(f"Warning: Unexpected landmarks shape {landmarks.shape} for video {filename}")
                                        landmarks = np.zeros((TOTAL_LANDMARKS, COORDINATES))

                                    # Append to dataset and labels
                                    all_datasets.append(landmarks)
                                    all_labels.append(sign_folder.split('.')[1].strip()) 

                                cap.release()

            # Save backup every 7 folders
            if folder_count % 7 == 0:
                np.save(f'backup_{folder_count}_pose_landmarks_dataset.npy', np.array(all_datasets))
                np.save(f'backup_{folder_count}_pose_landmarks_labels.npy', np.array(all_labels))

    # Save final dataset and labels
    np.save('final_pose_landmarks_dataset.npy', np.array(all_datasets))
    np.save('final_pose_landmarks_labels.npy', np.array(all_labels))

    # Cleanup
    pose.close()
    face_mesh.close()
    hands.close()

In [4]:
process_videos('/kaggle/input/include/')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1730536027.231769      74 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730536027.260135      74 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing videos in 87. hot:   0%|          | 0/21 [00:00<?, ?it/s]W0000 00:00:1730536027.297481      76 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730536027.335444      77 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1730536027.372827      68 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabl

Processing videos in 87. hot:  62%|██████▏   | 13/21 [01:26<00:53,  6.73s/it]

Processing videos in 87. hot:  67%|██████▋   | 14/21 [01:33<00:47,  6.85s/it]

Processing videos in 88. cold:  30%|███       | 6/20 [00:39<01:28,  6.30s/it]

Processing videos in 88. cold:  50%|█████     | 10/20 [01:06<01:03,  6.33s/it]

...

Processing videos in 51. Brown:  10%|█         | 2/20 [00:14<02:13,  7.40s/it]

Processing videos in 51. Brown:  55%|█████▌    | 11/20 [01:17<01:03,  7.09s/it]

Processing videos in 51. Brown:  60%|██████    | 12/20 [01:24<00:56,  7.05s/it]

Processing videos in 51. Brown:  85%|████████▌ | 17/20 [01:55<00:19,  6.34s/it]

Processing videos in 48. Green: 100%|██████████| 19/19 [02:11<00:00,  6.92s/it]
